In [1]:
import json
import gzip

import pandas as pd
from pathlib import Path
pd.set_option('max_colwidth',300)
from pprint import pprint

In [2]:
train_files = sorted(Path('../resources/data/python/final/jsonl/train').glob('**/*.gz'))
val_files = sorted(Path('../resources/data/python/final/jsonl/valid').glob('**/*.gz'))
test_files = sorted(Path('../resources/data/python/final/jsonl/test').glob('**/*.gz'))

In [3]:
print(len(train_files), len(val_files), len(test_files))

14 1 1


In [7]:
def remove(li):
    return [ele for ele in li if len(ele) > 0 and ele[0]!="#"]

def remove_inline_comments(files):
    all_data = []
    for f in files:
        with gzip.open(f, 'r') as ff:
            each = []
            data = ff.readlines()
            for i in data:
                new = {}
                loaded = json.loads(i)
                code_tokens = remove(loaded["code_tokens"])
                new["code"] = " ".join(code_tokens)
                docstring_tokens = loaded["docstring_tokens"]
                new["docstring"] = " ".join(docstring_tokens)
                each.append(new)
            all_data.extend(each)
    print (len(all_data))
    df = pd.DataFrame.from_records(all_data)
    return df

In [8]:
train = remove_inline_comments(train_files)
val = remove_inline_comments(val_files)
test = remove_inline_comments(test_files)

412178
23107
22176


In [9]:
train.head(1)

,code,docstring
0,"def train ( train_dir , model_save_path = None , n_neighbors = None , knn_algo = 'ball_tree' , verbose = False ) : X = [ ] y = [ ] for class_dir in os . listdir ( train_dir ) : if not os . path . isdir ( os . path . join ( train_dir , class_dir ) ) : continue for img_path in image_files_in_folde...",Trains a k - nearest neighbors classifier for face recognition .


In [10]:
train["code"][0]

'def train ( train_dir , model_save_path = None , n_neighbors = None , knn_algo = \'ball_tree\' , verbose = False ) : X = [ ] y = [ ] for class_dir in os . listdir ( train_dir ) : if not os . path . isdir ( os . path . join ( train_dir , class_dir ) ) : continue for img_path in image_files_in_folder ( os . path . join ( train_dir , class_dir ) ) : image = face_recognition . load_image_file ( img_path ) face_bounding_boxes = face_recognition . face_locations ( image ) if len ( face_bounding_boxes ) != 1 : if verbose : print ( "Image {} not suitable for training: {}" . format ( img_path , "Didn\'t find a face" if len ( face_bounding_boxes ) < 1 else "Found more than one face" ) ) else : X . append ( face_recognition . face_encodings ( image , known_face_locations = face_bounding_boxes ) [ 0 ] ) y . append ( class_dir ) if n_neighbors is None : n_neighbors = int ( round ( math . sqrt ( len ( X ) ) ) ) if verbose : print ( "Chose n_neighbors automatically:" , n_neighbors ) knn_clf = neighb

In [11]:
train["docstring"][0]

'Trains a k - nearest neighbors classifier for face recognition .'

In [12]:
train.to_csv("train_wo_inline.csv")
val.to_csv("val_wo_inline.csv")
test.to_csv("test_wo_inline.csv")